In [1]:
!git clone https://github.com/MishaelTech/series_analyzing_system.git

Cloning into 'series_analyzing_system'...
remote: Enumerating objects: 283, done.
remote: Counting objects: 100% (283/283), done.
remote: Compressing objects: 100% (262/262), done.
remote: Total 283 (delta 24), reused 274 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (283/283), 2.93 MiB | 15.45 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [8]:
!cd series_analyzing_system && pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00


In [21]:
pip install --upgrade evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: evaluate
    Found existing installation: evaluate 0.4.2
    Uninstalling evaluate-0.4.2:
      Successfully uninstalled evaluate-0.4.2


In [28]:
!cd series_analyzing_system && python gradio_app.py

2024-10-04 08:47:14.980545: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-04 08:47:15.004551: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-04 08:47:15.011698: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-04 08:47:15.034776: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-04 08:47:16.458945: W tensorflow/compiler/tf2

In [5]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from datasets import Dataset

# Load Dataset


In [ ]:
data_path="/content/series_analyzing_system/data/jutsus.jsonl"
df=pd.read_json(data_path,lines=True)
df.head()

,jutsu_name,jutsu_type,jutsu_description
0,10 Hit Combo,Taijutsu,Lars punches the opponent before striking them...
1,Area Scanning Technique,Ninjutsu,By touching a solid surface with their hand or...
2,Aqua Shoot,Ninjutsu,The user gathers a small orb of water and kick...
3,Armour-Eater,Ninjutsu,"The user melts pieces of their armour, creatin..."
4,Armageddon Countdown Clock,Ninjutsu,"After performing the hand seal, a special cloc..."


In [ ]:
# Have few jutsu_type name using function
def simplify_jutsu(jutsu):
    if "Genjutsu" in jutsu:
        return "Genjutsu"
    if "Ninjutsu" in jutsu:
        return "Ninjutsu"
    if "Taijutsu" in jutsu:
        return "Taijutsu"

In [ ]:
df["jutsu_type_simplified"]=df["jutsu_type"].apply(simplify_jutsu)
df.head()

,jutsu_name,jutsu_type,jutsu_description,jutsu_type_simplified
0,10 Hit Combo,Taijutsu,Lars punches the opponent before striking them...,Taijutsu
1,Area Scanning Technique,Ninjutsu,By touching a solid surface with their hand or...,Ninjutsu
2,Aqua Shoot,Ninjutsu,The user gathers a small orb of water and kick...,Ninjutsu
3,Armour-Eater,Ninjutsu,"The user melts pieces of their armour, creatin...",Ninjutsu
4,Armageddon Countdown Clock,Ninjutsu,"After performing the hand seal, a special cloc...",Ninjutsu


In [ ]:
# How much name we have at each type
df["jutsu_type_simplified"].value_counts()

,count
jutsu_type_simplified,
Ninjutsu,2258
Taijutsu,398
Genjutsu,101


In [ ]:
# Name and description in one column called text
df["text"]=df["jutsu_name"]+". "+df["jutsu_description"]
df["jutsu"]=df["jutsu_type_simplified"]
df.head()

,jutsu_name,jutsu_type,jutsu_description,jutsu_type_simplified,text,jutsu
0,10 Hit Combo,Taijutsu,Lars punches the opponent before striking them...,Taijutsu,10 Hit Combo. Lars punches the opponent before...,Taijutsu
1,Area Scanning Technique,Ninjutsu,By touching a solid surface with their hand or...,Ninjutsu,Area Scanning Technique. By touching a solid s...,Ninjutsu
2,Aqua Shoot,Ninjutsu,The user gathers a small orb of water and kick...,Ninjutsu,Aqua Shoot. The user gathers a small orb of wa...,Ninjutsu
3,Armour-Eater,Ninjutsu,"The user melts pieces of their armour, creatin...",Ninjutsu,Armour-Eater. The user melts pieces of their a...,Ninjutsu
4,Armageddon Countdown Clock,Ninjutsu,"After performing the hand seal, a special cloc...",Ninjutsu,Armageddon Countdown Clock. After performing t...,Ninjutsu


In [ ]:
# Taking only the text and jutsu
df=df[["text","jutsu"]]
# Drop missing values
df=df.dropna()

In [ ]:
df.head()

,text,jutsu
0,10 Hit Combo. Lars punches the opponent before...,Taijutsu
1,Area Scanning Technique. By touching a solid s...,Ninjutsu
2,Aqua Shoot. The user gathers a small orb of wa...,Ninjutsu
3,Armour-Eater. The user melts pieces of their a...,Ninjutsu
4,Armageddon Countdown Clock. After performing t...,Ninjutsu


In [ ]:
from bs4 import BeautifulSoup


# Creating a function called Cleaner to remove HTML Tags
class Cleaner:
    def __init__(self):
        pass

    #Putting line break after each paragraph
    def put_line_break(self,text):
        return text.replace("<\p> ", "<\p>\n")

    # Remove html tags using beautifulsoup
    def remove_html_tags(self,text):
        clean_text = BeautifulSoup(text, "lxml").text
        return clean_text


    def clean_html(self,text):
        clean_text = self.remove_html_tags(text)
        clean_text = self.put_line_break(clean_text)
        clean_text = clean_text.strip()
        return clean_text

In [ ]:
text_column_name="text"
label_column_name="jutsu"

In [ ]:
# Clean the text
cleaner=Cleaner()
df["text_cleaned"]=df[text_column_name].apply(cleaner.clean_html)

<ipython-input-18-a2ba18b7fb3d>:15: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  clean_text = BeautifulSoup(text, "lxml").text


In [ ]:
df.head(2)

,text,jutsu,text_cleaned
0,10 Hit Combo. Lars punches the opponent before...,Taijutsu,10 Hit Combo. Lars punches the opponent before...
1,Area Scanning Technique. By touching a solid s...,Ninjutsu,Area Scanning Technique. By touching a solid s...


In [ ]:
# Input need to be numbered so we can tokenizied the input, so it can have the output as numbers
# Making each jutsu to be outputted as number e.g Taijutsu=0, Ninjutsu=1 , etc using labeling encoders from sklearn

# Label Encoder
label_encoder=preprocessing.LabelEncoder()
label_encoder.fit(df[label_column_name].tolist())

LabelEncoder()

In [ ]:
# One line for loop that return a dict
label_dict = {index:label_name for index, label_name in enumerate(label_encoder.__dict__['classes_'].tolist())}
label_dict

# One line for loop that return a list
# labels_list = [(index, label_name) for index,label_name in enumerate(label_encoder.__dict__['classes_'].tolist())]
# labels_list

{0: 'Genjutsu', 1: 'Ninjutsu', 2: 'Taijutsu'}

In [ ]:
# Transform the jutsu to numerical type
df["label"]=label_encoder.transform(df[label_column_name].tolist())
df.head()

,text,jutsu,text_cleaned,label
0,10 Hit Combo. Lars punches the opponent before...,Taijutsu,10 Hit Combo. Lars punches the opponent before...,2
1,Area Scanning Technique. By touching a solid s...,Ninjutsu,Area Scanning Technique. By touching a solid s...,1
2,Aqua Shoot. The user gathers a small orb of wa...,Ninjutsu,Aqua Shoot. The user gathers a small orb of wa...,1
3,Armour-Eater. The user melts pieces of their a...,Ninjutsu,Armour-Eater. The user melts pieces of their a...,1
4,Armageddon Countdown Clock. After performing t...,Ninjutsu,Armageddon Countdown Clock. After performing t...,1


# Traing & Testing of the dataset



In [ ]:
# 80:20 for training and testing of the dataset
# 80 of the jutsu in the training. We use train_test_split
test_size=0.2
train_df,test_df=train_test_split(df,
                                  test_size=test_size,
                                  stratify=df["label"],) # stratify makes sure that each of the classes that 80% is in training and 20% is in the test

In [ ]:
train_df["jutsu"].value_counts()

,count
jutsu,
Ninjutsu,1806
Taijutsu,318
Genjutsu,81


In [ ]:
# Tokenize output and the text
model_name="distilbert/distilbert-base-uncased"

In [ ]:
# We will use lamma for chatbot
tokenizer=AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Function that tokenizes the text
def preprocess_function(tokenizer,examples):
    return tokenizer(examples["text_cleaned"], truncation=True) # If it something above the 512 token it truncates so it wouldn't break the model

In [ ]:
# Run it on the whole dataset
# Convert Pandas (df) to a hugging face dataset
train_dataset=Dataset.from_pandas(train_df)
test_dataset=Dataset.from_pandas(test_df)

# tokenize the dataset
tokenized_train_dataset=train_dataset.map(lambda examples: preprocess_function(tokenizer,examples), batched=True)

tokenized_test_dataset=test_dataset.map(lambda examples: preprocess_function(tokenizer,examples), batched=True)

Map:   0%|          | 0/2205 [00:00<?, ? examples/s]

Map:   0%|          | 0/552 [00:00<?, ? examples/s]